<a href="https://colab.research.google.com/github/EleniHaylemeskel/Week-3-AI-Tools-Assignment/blob/main/Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# PART 2 - TASK 1: Classical Machine Learning with Scikit-learn
# Dataset: Iris Species
# Goal: Preprocess → Train Decision Tree → Evaluate


# --- Import Libraries ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix

# --- Step 1: Load Dataset ---
df = pd.read_csv("Iris.csv")

# Remove unnamed or unnecessary columns
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
df.columns = df.columns.str.strip()

print("Dataset Loaded Successfully!")
print(df.head())

# --- Step 2: Check & Handle Missing Values ---
print("\nMissing Values:\n", df.isnull().sum())

# Fill missing numeric values with median (if any)
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
for col in num_cols:
    if df[col].isnull().any():
        df[col].fillna(df[col].median(), inplace=True)
        print(f"Filled missing values in column: {col}")

# --- Step 3: Encode Label Column ---
label_col = "Species"
le = LabelEncoder()
df[label_col] = le.fit_transform(df[label_col])

print("\nLabel Encoding Completed!")
print("Classes:", le.classes_)

# --- Step 4: Prepare Features and Labels ---
# Remove ID column if present
feature_cols = [col for col in df.columns if col not in ["Species", "Id"]]

X = df[feature_cols]
y = df[label_col]

# --- Step 5: Split Train & Test ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("\nData Split Completed!")
print("Training Shape:", X_train.shape)
print("Test Shape:", X_test.shape)

# --- Step 6: Train the Decision Tree Classifier ---
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

print("\nModel Training Completed!")

# --- Step 7: Predictions ---
y_pred = clf.predict(X_test)

# --- Step 8: Evaluation ---
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="macro")
recall = recall_score(y_test, y_pred, average="macro")

print("\n=== MODEL PERFORMANCE ===")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision (Macro): {precision:.4f}")
print(f"Recall (Macro): {recall:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nTask 1 Completed Successfully! 🎉")

In [ ]:
# PART 2 – TASK 2: Deep Learning with TensorFlow (MNIST CNN)
# Dataset: MNIST Handwritten Digits
# Goal: Build a Model → Test Accuracy → Visualize the Model

import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np


# 1. Load and Preprocess Data

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize pixel values (0–255 → 0–1)
x_train = x_train / 255.0
x_test = x_test / 255.0

# Reshape for CNN: (batch, width, height, channels)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)


# 2. Build CNN Model

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # 10 digits
])

# Compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# 3. Train CNN Model

history = model.fit(
    x_train, y_train,
    epochs=5,
    batch_size=64,
    validation_split=0.1
)


# 4. Evaluate on Test Set

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print("\nTest Accuracy:", test_acc)


# 5. Visualize Predictions (5 Sample Images)

predictions = model.predict(x_test)

def plot_sample(i):
    plt.imshow(x_test[i].reshape(28,28), cmap="gray")
    plt.title(f"Prediction: {np.argmax(predictions[i])}\nActual: {y_test[i]}")
    plt.axis("off")
    plt.show()

# Show 5 random samples
for i in np.random.randint(0, len(x_test), 5):
    plot_sample(i)


In [5]:
# Task 3: NLP with spaCy
# Text Data: User reviews from Amazon Product Reviews
# Goal: Extract product names/brands using NER + sentiment rules


import spacy

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

# --- Sample Amazon-style reviews ---
reviews = [
    "I love my new Samsung Galaxy phone. The camera quality is amazing!",
    "The Apple AirPods are overpriced and the battery dies quickly.",
    "This Sony speaker has incredible bass. Totally worth the price!",
    "I bought a Dell laptop and it keeps overheating. Not happy with it.",
    "The Nike running shoes are very comfortable and durable."
]

# --- Rule-based sentiment analyzer ---
positive_keywords = ["love", "great", "amazing", "incredible", "worth", "comfortable", "durable"]
negative_keywords = ["bad", "overpriced", "dies", "poor", "overheating", "not happy"]

def rule_based_sentiment(text):
    text_lower = text.lower()
    score = 0

    for word in positive_keywords:
        if word in text_lower:
            score += 1

    for word in negative_keywords:
        if word in text_lower:
            score -= 1

    if score > 0:
        return "Positive"
    elif score < 0:
        return "Negative"
    else:
        return "Neutral"

# --- Process each review ---
for review in reviews:
    doc = nlp(review)

    print("\nReview:", review)

    # Extract product-related entities
    print("Named Entities (Product/Brand):")
    for ent in doc.ents:
        if ent.label_ in ["ORG", "PRODUCT"]:
            print(f"  - {ent.text} ({ent.label_})")

    # Sentiment result
    sentiment = rule_based_sentiment(review)
    print("Sentiment:", sentiment)


Review: I love my new Samsung Galaxy phone. The camera quality is amazing!
Named Entities (Product/Brand):
  - Samsung Galaxy (ORG)
Sentiment: Positive

Review: The Apple AirPods are overpriced and the battery dies quickly.
Named Entities (Product/Brand):
Sentiment: Negative

Review: This Sony speaker has incredible bass. Totally worth the price!
Named Entities (Product/Brand):
  - Sony (ORG)
Sentiment: Positive

Review: I bought a Dell laptop and it keeps overheating. Not happy with it.
Named Entities (Product/Brand):
  - Dell (ORG)
Sentiment: Negative

Review: The Nike running shoes are very comfortable and durable.
Named Entities (Product/Brand):
  - Nike (ORG)
Sentiment: Positive


In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(28,28)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/ops/nn.py:944: UserWarning: You are using a softmax over axis -1 of a tensor of shape (32, 28, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/losses/losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(32, 28, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(32,), output.shape=(32, 28, 1)

In [7]:
import tensorflow as tf

# Correct input shape: flatten 28x28 into 784
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')   # 10 digit classes
])

# Correct loss for integer labels
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the repaired model
model.fit(x_train, y_train, epochs=5, batch_size=64)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8506 - loss: 0.5139
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9632 - loss: 0.1247
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9761 - loss: 0.0780
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9817 - loss: 0.0590
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9856 - loss: 0.0453
